In [6]:
import tweepy as tw
import pandas as pd
import requests
import json
from textblob import TextBlob
import re
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from bs4 import BeautifulSoup
import requests
import pprint
import pymongo
from sqlalchemy import func
from nltk.stem import WordNetLemmatizer

import json
import csv
from textblob import TextBlob
import string
import os
import time



In [16]:
# The consumer keys can be found on your application's Details
# page located at https://dev.twitter.com/apps (under "OAuth settings")
access_token = "114010579-jAnTPiNwF9krPFR68yX61Qqtw2NluLs8dNQaoi7E"
access_token_secret = "eude9xgmDMSvLIaylw2tNJTd0LXd2i7wfKZFA9zWKNqIp"
consumer_key = "j2BhP2Q3LFCzpPQ5o0Xx4GuY7"
consumer_secret = "vkcmHL1U71cpqdn1LBl7GY4cdJIkvFotKAyZeChIuHBoq5aQ2j"

master_list = (["#NotToday", 
              "#TheLongNight.", 
              "#GameofThrones",  
              "#Dracarys",  
              "#GOT", 
              "#GOTS8", 
              "#ForTheThrone", 
              "#DaenerysTargaryen", 
              "#JonSnow", 
              "#NightKing", 
              "#CerseiLannister", 
              "#AryaStark", 
              "#JaimeLannister", 
              "#TyrionLannister", 
              "#SansaStark",
              "#BranStark", 
              "#BrienneOfTarth", 
              "#DavosSeaworth", 
              "#EuronGreyjoy", 
              "#JorahMormont", 
              "#Greyworm", 
              "#Melisandre", 
              "#Missandei", 
              "#SamwellTarly", 
              "#TheonGreyjoy", 
              "#Varys", 
              "#TheHound"
             ])
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)



In [22]:
def scraptweets(search_words, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)
# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
# Begin scraping the tweets individually:
        noTweets = 0
for tweet in tweet_list:
# Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
try:
                text = tweet.retweeted_status.full_text
#             except AttributeError:  # Not a Retweet
#                 text = tweet.full_text
# Add the 11 variables to the empty list - ith_tweet:
                ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
# Append to dataframe - db_tweets
                db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
                noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) #15 minute sleep time
# Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
# Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_sahkprotests_tweets.csv'
# Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

IndentationError: unindent does not match any outer indentation level (<ipython-input-22-fbc943473f53>, line 62)

In [13]:
search_words = '#DaenerysTargaryen OR "Daenerys Targaryen"'
char_name = "Daenerys Targaryen"

dates = [["2019-05-19", "2019-05-20"], 
         ["2019-05-20", "2019-05-21"] 
        ]

date_since = ""
date_until = ""
tweet_text = []
tweet_timestamp = []
tweet_date = []
tweet_char = []
tweet_sen = []
tweet_pol = []
tweet_sub = []

for date_list in dates:
    date_since = date_list[0]
    date_until = date_list[1]
    tweets = tw.Cursor(api.search,
            q=search_words,
            lang="en",
            since=date_since, until=date_until, encode="utf-8").items(500)
    
    for tweet in tweets:
        tweet_date.append(tweet.created_at.strftime('%m/%d/%Y'))
        tweet_timestamp.append(tweet.created_at.strftime('%m/%d/%Y %H:%M:%S'))
    
        analysis = TextBlob(tweet.text)

        senti = ""
        polarity = 0.0
        subjectivity = 0.0
    
        if analysis.sentiment[0]>0:
           senti = "Pos"
        elif analysis.sentiment[0]<0:
           senti = "Neg"
        else:
           senti = "Neu"
    
        polarity = float(analysis.sentiment[0])
        subjectivity = float(analysis.sentiment[1])

        tweet_text.append(tweet.text)
        tweet_char.append(char_name)
        tweet_pol.append(polarity)
        tweet_sen.append(senti)
        tweet_sub.append(subjectivity)



In [14]:
df = pd.DataFrame({
  "text": tweet_text,
  "date" : tweet_date,
  "char" : tweet_char,
  "sentiment" : tweet_sen,
  "polarity" : tweet_pol,
  "subjectivity" : tweet_sub,
  "timestamp" : tweet_timestamp
})

df = df.dropna()
stemmer = WordNetLemmatizer()

for index, row in df.iterrows():
    # Remove all the special characters
    test_preproc = re.sub(r'\W', ' ', str(row["text"]))

    # remove all single characters
    test_preproc = re.sub(r'\s+[a-zA-Z]\s+', ' ', test_preproc)

    # Remove single characters from the start
    test_preproc = re.sub(r'\^[a-zA-Z]\s+', ' ', test_preproc) 

    # Substituting multiple spaces with single space
    test_preproc = re.sub(r'\s+', ' ', test_preproc, flags=re.I)

    # Removing prefixed 'b'
    test_preproc = re.sub(r'^b\s+', '', test_preproc)

    # Converting to Lowercase
    test_preproc = test_preproc.lower()
    
    # Lemmatization
    test_preproc = test_preproc.split()

    test_preproc = [stemmer.lemmatize(word) for word in test_preproc]
    comb_string = ' '.join(test_preproc)
    df.loc[index, "Preprocess"] = comb_string



In [15]:
df.to_csv("DaenerysTargaryen_19_21.csv", index=False)